In [2]:
import yfinance as yf
import pandas as pd

List of Tickers

In [3]:
tickers = ["ITSA4.SA", "ITUB4.SA"]

In [ ]:
data = yf.download(tickers, start="2024-01-01", end="2025-10-01")
# data.index = pd.to_datetime(data.index)

print(data.head())

In [13]:
keys = ["ticker", "cumulative_return_period","return_std_deviation"]
metrics = {key: [] for key in keys}

for ticker in tickers:
    close_values = data[('Close', ticker)]
    close_values = close_values.reset_index(drop=True)

    cumulative_return_period = (close_values.iloc[-1])-(close_values.iloc[0])
    daily_return = close_values.diff()
    return_std_deviation = daily_return.std()

    metrics["ticker"].append(ticker)
    metrics["cumulative_return_period"].append(cumulative_return_period)
    metrics["return_std_deviation"].append(return_std_deviation)

df = pd.DataFrame(metrics)
print(metrics)
print(df)

{'ticker': ['ITSA4.SA', 'ITUB4.SA'], 'cumulative_return_period': [np.float64(3.209174156188965), np.float64(12.572225570678711)], 'return_std_deviation': [np.float64(0.11099721164765151), np.float64(0.37594936538943047)]}
     ticker  cumulative_return_period  return_std_deviation
0  ITSA4.SA                  3.209174              0.110997
1  ITUB4.SA                 12.572226              0.375949
